# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.243/11383/1 Dashboard: http://10.20.0.243:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
df.head()

,id,name,x,y
0,943,Hannah,-0.981482,-0.764403
1,984,Xavier,-0.958945,-0.175703
2,973,Kevin,0.338215,-0.170472
3,998,Norbert,0.722924,0.135131
4,979,Kevin,-0.714298,-0.052212


Let's look at some statistics on the data

In [7]:
df.describe().compute()

,id,x,y
count,2.592000e+06,2.592000e+06,2.592000e+06
mean,1.000047e+03,-1.320420e-04,2.397678e-04
std,3.161170e+01,5.772522e-01,5.772048e-01
min,8.470000e+02,-9.999981e-01,-9.999991e-01
25%,9.790000e+02,-4.924321e-01,-4.929244e-01
50%,1.000000e+03,6.665752e-03,8.185026e-03
75%,1.022000e+03,5.037702e-01,5.078510e-01
max,1.158000e+03,9.999997e-01,9.999994e-01


# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
973,Bob,-0.19384293430895935
1019,Sarah,-0.7880814569856256
1013,Wendy,0.06998129545406373
987,Bob,-0.8175034384458173
1036,Yvonne,-0.5869418302153475
941,Norbert,-0.2111612548419346
1041,Sarah,0.2981630356537901
989,Norbert,0.4504568831474469
1031,Xavier,-0.2982059427905821


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

,id,name,x,y
0,943,Hannah,-0.981482,-0.764403
1,984,Xavier,-0.958945,-0.175703
2,973,Kevin,0.338215,-0.170472
3,998,Norbert,0.722924,0.135131
4,979,Kevin,-0.714298,-0.052212


Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((subgraph_callable, (<function pandas_read_text at 0x7f0b224f6ca0>, <function _make_parser_function.<locals>.parser_f at 0x7f0b346d6550>, (<function read_block_from_file at 0x7f0b204ba310>, <OpenFile '/home/travis/build/dask/dask-examples/dataframes/data/2000-01-07.csv'>, 0, 64000000, b'\n'), b'id,name,x,y\n', {}, {'id': dtype('int64'), 'name': dtype('O'), 'x': dtype('float64'), 'y': dtype('float64')}, ['id', 'name', 'x', 'y'], False, False, None), 'read-csv-e5cce6384a7930ff398a0d69dacef588'))
kwargs:    {}
Exception: ValueError('Length mismatch: Expected axis has 3 elements, new values have 4 elements')



ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

,id,name,x,y
0,943,Hannah,-0.981482,-0.764403
1,984,Xavier,-0.958945,-0.175703
2,973,Kevin,0.338215,-0.170472
3,998,Norbert,0.722924,0.135131
4,979,Kevin,-0.714298,-0.052212


In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

,id,name,x,y


In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

,id,name,x,y
0,973,Bob,-0.193843,NaN
1,1019,Sarah,-0.788081,NaN
2,1013,Wendy,0.069981,NaN
3,987,Bob,-0.817503,NaN
4,1036,Yvonne,-0.586942,NaN


# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-29b80acc-6374-43f3-a945-e329539e5964')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

,id,name,x,y
0,943,Hannah,-0.981482,-0.764403
1,984,Xavier,-0.958945,-0.175703
2,973,Kevin,0.338215,-0.170472
3,998,Norbert,0.722924,0.135131
4,979,Kevin,-0.714298,-0.052212
...,...,...,...,...
86395,996,Alice,0.339934,-0.953161
86396,1008,Quinn,0.517841,0.215181
86397,1023,Sarah,0.468068,-0.683689
86398,965,Victor,-0.148390,0.023047


Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-13.csv',
 'data/2000-01-02.csv',
 'data/2000-01-24.csv',
 'data/2000-01-04.csv',
 'data/2000-01-09.csv',
 'data/2000-01-17.csv',
 'data/2000-01-06.csv',
 'data/2000-01-08.csv',
 'data/2000-01-11.csv',
 'data/2000-01-28.csv',
 'data/2000-01-29.csv',
 'data/2000-01-18.csv',
 'data/2000-01-16.csv',
 'data/2000-01-05.csv',
 'data/2000-01-30.csv',
 'data/2000-01-20.csv',
 'data/2000-01-19.csv',
 'data/2000-01-21.csv',
 'data/2000-01-27.csv',
 'data/2000-01-12.csv',
 'data/2000-01-01.csv',
 'data/2000-01-07.csv',
 'data/2000-01-10.csv',
 'data/2000-01-25.csv',
 'data/2000-01-14.csv',
 'data/2000-01-26.csv',
 'data/2000-01-23.csv',
 'data/2000-01-03.csv',
 'data/2000-01-15.csv',
 'data/2000-01-22.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-84e8c48a-8c52-4bb9-aedf-a6bb3598be4d'),
 Delayed('read_data-d6285f31-ac65-4370-947e-7e78e68bc7bc'),
 Delayed('read_data-b0656d4f-398a-48cc-848c-da7464c66224'),
 Delayed('read_data-cedb838f-d850-47b5-84ce-e6dcf8144d07'),
 Delayed('read_data-5f71b22f-2214-4e4b-a427-4325722fbb9e'),
 Delayed('read_data-2d434858-930b-4d66-84cf-978bc3e8d3d5'),
 Delayed('read_data-9922a315-7d79-424f-a32d-45aa5999598a'),
 Delayed('read_data-bda3bff0-3011-4f06-95f4-fbbb0b499486'),
 Delayed('read_data-b4ae1366-f90e-43c5-bbaf-6ee35d5e5e93'),
 Delayed('read_data-1065ddb8-2c39-4d32-bf20-b24842b0f1fb'),
 Delayed('read_data-4af683e7-c21f-4851-96bb-dfc58b937f33'),
 Delayed('read_data-759c7462-4419-4d66-b4fc-05b2f44df372'),
 Delayed('read_data-d6589058-83f4-41e5-a24e-890f404df125'),
 Delayed('read_data-b9abedf6-5515-453b-9199-057fc03b8b69'),
 Delayed('read_data-ae113bc4-8d66-4f02-a184-a0ff5ef475cb'),
 Delayed('read_data-4fc19cb0-d8b8-4ce2-a57b-7afdcb268a34'),
 Delayed('read_data-402feed5-ef46-4d80-9

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.from_delayed). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

,id,name,x,y
0,1062,Laura,0.637772,-0.709460
1,1017,Patricia,-0.476572,0.881341
2,1029,Alice,-0.873784,-0.052437
3,975,Frank,0.452187,0.244706
4,917,Xavier,0.594187,-0.050097


In [26]:
df.describe().compute()

/home/travis/miniconda/envs/test/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


,id,x,y
count,2.505600e+06,2.505600e+06,2.419200e+06
mean,1.000045e+03,-1.912869e-04,1.649934e-04
std,3.161118e+01,5.772524e-01,5.772506e-01
min,8.470000e+02,-9.999981e-01,-9.999991e-01
25%,9.790000e+02,-4.924321e-01,-4.929244e-01
50%,1.000000e+03,6.665752e-03,8.185026e-03
75%,1.022000e+03,5.037702e-01,5.078510e-01
max,1.158000e+03,9.999997e-01,9.999994e-01


## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.